In [4]:
import sys
sys.path.append('/home/marco/Desktop/units/Thesis/PPCon-evo/evodenss')
import pandas as pd
from evodenss.dataset.dataset_loader import FloatDataset, MyDataset
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from torchvision.datasets import CIFAR10
import torch


#data = pd.read_csv('/home/marco/Desktop/units/Thesis/PPCon/ds/BBP700/float_ds_sf_removed.csv')

In [ ]:
#let's merge the two datasets
dataset = pd.concat([dataset1.df, dataset2.df], axis=1, ignore_index=True)
#drop the first 4 rows
dataset = dataset.drop(dataset.columns[[0,1,2,3]], axis=0)
#transpose the dataset
dataset = dataset.transpose()

dataset.iloc[30,23]

In [69]:
training_dataset = FloatDataset(path_df='/home/marco/Desktop/units/Thesis/PPCon/ds/BBP700/float_ds_sf_train.csv')
testing_dataset = FloatDataset(path_df='/home/marco/Desktop/units/Thesis/PPCon/ds/BBP700/float_ds_sf_test.csv')

In [ ]:
def get_manageable_data(dataset):
    """
    This function is used to get the data in a format that can be used by the model, i.e. 
    the data is returned in a tensor of shape (n_samples, n_time_steps, n_features = 4)
    Dataset is the output of __getitem__ method of the FloatDataset class, that uses the
    from_string_to_tensor method to convert the string to a single entry of the dataset.
    """
    data = dataset[0][4].unsqueeze(0)
    for i in range(1, len(dataset)):
        data = torch.cat((data, dataset[i][4].unsqueeze(0)),0 )
    data = data.unsqueeze(2)
    for j in range(5,8):
        data2 = dataset[0][j].unsqueeze(0)
        for i in range(1, len(dataset)):
            data2 = torch.cat((data2, dataset[i][j].unsqueeze(0)),0 )
        data2 = data2.unsqueeze(2)
        data = torch.cat((data, data2),2)
    return data

train_data = get_manageable_data(training_dataset)
test_data = get_manageable_data(testing_dataset)

test_data, val_data = train_test_split(test_data, test_size=0.5, random_state=42)

print(train_data.shape, test_data.shape, val_data.shape)


In [100]:
#let's switch the last two dimensions of the data
#train_data = train_data.permute(0,2,1)
#test_data = test_data.permute(0,2,1)
#val_data = val_data.permute(0,2,1)



torch.save(train_data, '/home/marco/Desktop/units/PPCon-evo/evodenss/data/ds/BBP700/train_data.pt')
torch.save(test_data, '/home/marco/Desktop/units/PPCon-evo/evodenss/data/ds/BBP700/test_data.pt')
torch.save(val_data, '/home/marco/Desktop/units/PPCon-evo/evodenss/data/ds/BBP700/val_data.pt')




In [97]:
train_data.shape, test_data.shape, val_data.shape

(torch.Size([3980, 4, 200]),
 torch.Size([498, 4, 200]),
 torch.Size([498, 4, 200]))

In [5]:
a = torch.load('/home/marco/Desktop/units/PPCon-evo/evodenss/data/ds/BBP700/train_data.pt')
a.shape

torch.Size([3980, 4, 200])

In [107]:

class MyDataset(torch.utils.data.Dataset):
    def __init__(self, path_df=None, path_tensor = None, transform=None):
        super(MyDataset, self).__init__()
        if path_df is not None:
            self.path_df = path_df
            self.df = pd.read_csv(self.path_df)
            self.data = self.df.iloc[:, :-1].values
            self.targets = self.df.iloc[:, -1].values

        elif path_tensor is not None:
            self.df = torch.load(path_tensor)
            self.data = self.df[:,:-1, :]
            self.targets = self.df[:,-1, :]
        else:
            raise Exception("Paths should be given as input to initialize the Float class.")
        self.transform = transform
     
        

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.data)
    
train_data = MyDataset(path_tensor='/home/marco/Desktop/units/PPCon-evo/evodenss/data/ds/BBP700/test_data.pt')
train_data.data.shape

torch.Size([498, 3, 200])

In [19]:
import unittest

import numpy as np
from parameterized import parameterized
import sys
sys.path.append('..')

from evodenss.config.pydantic import DataSplits, DownstreamTrain, EvoTest, Labelled, Validation, init_context
from evodenss.dataset.dataset_loader import DatasetProcessor, DatasetType
from evodenss.misc.constants import DEFAULT_SEED
from evodenss.misc.enums import LearningType
from evodenss.networks.transformers import LegacyTransformer


In [20]:
dataset_processor = DatasetProcessor(
            ssl_transformer=None,
            train_transformer=LegacyTransformer({}),
            test_transformer=LegacyTransformer({})
        )
with init_context({'learning_type': LearningType.supervised}):
    train_partition_ratio = 0.007
    validation_partition_ratio = 0.001
    test_partition_ratio = 0.002
    data_splits = DataSplits(
        unlabelled=None,
        labelled=Labelled(
            percentage=1,
            downstream_train=DownstreamTrain(
                partition_ratio=train_partition_ratio,
                amount_to_use=1.0,
                replacement=False),
            validation=Validation(
                partition_ratio=validation_partition_ratio,
                amount_to_use=1.0,
                replacement=False),
            evo_test=EvoTest(
                partition_ratio=test_partition_ratio,
                amount_to_use=1.0,
                replacement=False)
        )
    )
subset_dict = dataset_processor.load_partitioned_dataset(
        dataset_name="mnist",
        proportions=data_splits,
        seed=DEFAULT_SEED)


In [26]:
train_labelled_data = CIFAR10(
            root="data",
            train=True,
            download=True,
            transform=LegacyTransformer({})
        )

100.0%


Extracting data/cifar-10-python.tar.gz to data


In [27]:
train_labelled_data.data.shape


(50000, 32, 32, 3)

In [17]:
import torch
import torch.nn as nn

class LegacyNetwork(nn.Module):
    def __init__(self):
        super(LegacyNetwork, self).__init__()
        
        self.conv1d_1 = nn.Sequential(
            nn.Conv1d(in_channels=3, out_channels=36, kernel_size=5, stride=2, padding=0, bias=False),
            nn.ReLU()
        )
        
        self.conv1d_2 = nn.Sequential(
            nn.Conv1d(in_channels=36, out_channels=39, kernel_size=3, stride=1, padding=0),
            nn.Sigmoid()
        )
        
        self.conv1d_3 = nn.Sequential(
            nn.Conv1d(in_channels=39, out_channels=63, kernel_size=2, stride=2, padding=0),
            nn.ReLU()
        )
        
        self.conv1d_4 = nn.Conv1d(in_channels=63, out_channels=69, kernel_size=4, stride=1, padding=1)  # Padding = same
        self.conv1d_5 = nn.Conv1d(in_channels=69, out_channels=45, kernel_size=3, stride=1, padding=1)  # Padding = same
        self.conv1d_6 = nn.Conv1d(in_channels=45, out_channels=14, kernel_size=2, stride=2, padding=0, bias=False)

        self.fc_1 = nn.Sequential(
            nn.Flatten(start_dim=1, end_dim=-1),
            nn.Linear(in_features=322, out_features=200, bias=False),
            nn.ReLU()
        )

        self.fc_2 = nn.Sequential(
            nn.Linear(in_features=200, out_features=200, bias=True),
            nn.Softmax(dim=None)
        )

    def forward(self, x):
        x = self.conv1d_1(x)
        print(x.shape)
        x = self.conv1d_2(x)
        print(x.shape)
        x = self.conv1d_3(x)
        print(x.shape)
        x = self.conv1d_4(x)
        print(x.shape)
        x = self.conv1d_5(x)
        print(x.shape)
        x = self.conv1d_6(x)
        print(x.shape)
        x = self.fc_1(x)
        x = self.fc_2(x)
        return x

# Example of creating the model
model = LegacyNetwork()


In [18]:
batch_train = a[:123,:3,:]
batch_train.shape


torch.Size([123, 3, 200])

In [19]:
model(batch_train)

torch.Size([123, 36, 98])
torch.Size([123, 39, 96])
torch.Size([123, 63, 48])
torch.Size([123, 69, 47])
torch.Size([123, 45, 47])
torch.Size([123, 14, 23])


/home/marco/miniconda3/envs/evodenss/lib/python3.10/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor([[0.0046, 0.0051, 0.0047,  ..., 0.0053, 0.0052, 0.0050],
        [0.0046, 0.0051, 0.0047,  ..., 0.0053, 0.0052, 0.0050],
        [0.0046, 0.0051, 0.0047,  ..., 0.0053, 0.0052, 0.0050],
        ...,
        [0.0046, 0.0051, 0.0047,  ..., 0.0053, 0.0052, 0.0050],
        [0.0046, 0.0051, 0.0047,  ..., 0.0053, 0.0052, 0.0050],
        [0.0046, 0.0051, 0.0047,  ..., 0.0053, 0.0052, 0.0050]],
       grad_fn=<SoftmaxBackward0>)